# Project 4: Sequential Structure from Motion

### Due 4/3/2019

### Graduate Students: Our next reading is [Snavely, 2006](http://195.130.87.21:8080/dspace/bitstream/123456789/636/1/Photo%20tourism%20exploring%20photo%20collections%20in%203D.pdf).  We'll have a written report on this one: these methods papers aren't as good for discussions as I'd hoped.

## Problem Statement

You have now developed code that takes two photographs, finds matching locations in both, determines the relative motion between the cameras that took both photos, and solves for the 3D position of those points using epipolar geometry.  **The next (and final for our purposes) stage is to extend this analysis to more than two images**, such that we can build 3D models of objects on the ground with just about as much detail as we'd like.

## Adding the third photo
How do we add these additional photos?  To be perfectly honest, at this point it's mostly an exercise in match housekeeping: we've already developed most of the code that we need.  First, let's consider what we've got after matching our first two images, $I_1$ and $I_2$.  First, we have a set of keypoints in each image, associated with a set of matches.  These matches have been quality controlled twice:  first by the ratio test, then by RANSAC in conjunction with the recovery of the essential matrix.  Assuming that we've used our known camera matrix to convert our pixel-wise coordinates to generalized coordinates, let's call these keypoints $\mathbf{x}_1$ and $\mathbf{x}_2$.  In practice, we can drop all of those keypoints for which there is not an associated accepted match.  Then, for each of our kept matches, we have the essential matrix $E_{12}$, from which we can extract a pair of projection matrices $\mathbf{P}_1 = [\mathbf{I}|\mathbf{0}]$ and $\mathbf{P}_2 = [\mathbf{R}_2|\mathbf{t}_2]$.  Using these projection matrices, we generated 3D, world coordinate location of the corresponding features that showed up robustly in both images.  We'll call these coordinates $\mathbf{X}_{12}$.  

To add a third image $\mathbf{I}_3$ to the mix, consider that the situation outlined above is sort of analogous to the information that we have when we want to do pose estimation with ground control points: we have 3D world coordinates as well as the image coordinates of a set of points (a bunch of them, usually!), and we want to recover the camera pose.  The problem is that the feature generalized coordinates that we've computed are for $I_1$ and $I_2$, but not $I_3$.  Is this a big problem?  Of course not!  We can simply find $\mathbf{x}_3$ in $I_3$ that correspond to $\mathbf{x}_2$, the keypoints in the second image.  Then we identify these keypoints with the 3D poi nts $\mathbf{X}_{12}$.  Thus we have image coordinates of features in the third image and the corresponding world coordinates: we can now perform pose estimation, just as we did in Project 1.  

Of course there are a few minor caveats: first, we need to filter out spurious matches between $\mathbf{x}_2$ and $\mathbf{x}_3$.  To do this, we can utilize a tool that we already have: RANSAC estimation of the essential matrix.  Because $I_2$ and $I_3$ are related by epipolar geometry just as $I_1$ and $I_2$ are, we can use the same subroutine to compute the essential matrix $\mathbf{E}_{23}$, and (critically) identify and filter outliers, i.e. we'll discard matches that don't don't correspond to the consensus view of the essential matrix.  This also leads to the next caveat, namely that we need an initial guess (call it $P_3^0$) for pose estimation to converge properly.  Where should this initial guess come from?  The $\mathbf{E}_{23}$ provides a rotation given as if camera 2 were canonical, i.e. $\mathbf{P_2'}=[\mathbf{I}|\mathbf{0}]$, $\mathbf{P_3}'=[\mathbf{R}_3'|\mathbf{t}_3']$.  We'll call it $P_3'$.  We need to convert this projection matrix to a coordinate system in which $I_1$ (not $I_2$) is canonical.  Fortunately, this is easy:
$$
P_3 \approx P_2 P_3'.
$$
This $P_3$, is a an excellent initial guess for pose estimation (in principle, it's rotation matrix should actually be correct).  Note that the translation component is only good up to a constant: however, this isn't too problematic because its direction is close to correct, and any optimization just needs to perform what amounts to a line search (a univariate optimization problem) to find the correct scaling. 

Once we have a robust estimation of the third camera's pose, we can use it do point triangulation on the correspondences between $I_2$ and $I_3$ not associated with an already-known world coordinate point, which allows us to augment our 3D model with new points.  Additionally, we can perform triangulation with *3 views*, potentially improving our accuracy.  Moreover, we can apply the process above iteratively, adding more and more images to generate a highly featured 3D model from (for example) 360 degrees worth of view angles.  

## Application
**Generate code that performs the above process for a third image.  Apply it to one of the 3D image datasets that we generated in class.  Note that we will be collecting aerial imagery from drones as well.  Apply this method to a sequence of drone imagery as well.**  As a challenge, can you implement code that sequentially adds an arbitrary number of images?


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import piexif

def get_best_sift_matches(I_1, I_2, r=0.7):

    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
    kp1, des1 = sift.detectAndCompute(I_1, None)
    kp2, des2 = sift.detectAndCompute(I_2, None)

    matcher = cv2.BFMatcher()
    matches = matcher.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        # Compute the ratio between best match m, and second best match n here
        if m.distance < r * n.distance:
            good_matches.append(m)

    u1 = []
    u2 = []

    for g in good_matches:
        u1.append(kp1[g.queryIdx].pt)
        u2.append(kp2[g.trainIdx].pt)

    u1 = np.array(u1)
    u2 = np.array(u2)

    u1 = np.c_[u1, np.ones(u1.shape[0])]
    u2 = np.c_[u2, np.ones(u2.shape[0])]

    return u1, u2

In [12]:
def get_correspondence_points(I_1, img_name, u1, u2):
    h,w,d = I_1.shape
    exif = piexif.load(img_name)
    f = exif['Exif'][piexif.ExifIFD.FocalLengthIn35mmFilm]/36*w
    cu = w//2
    cv = h//2

    K_cam = np.array([[f,0,cu], [0,f,cv], [0,0,1]])
    K_inv = np.linalg.inv(K_cam)
    x1 = u1 @ K_inv.T
    x2 = u2 @ K_inv.T 
    
    # compute E and inliers
    E, inliers = cv2.findEssentialMat(x1[:, :2], x2[:, :2], np.eye(3), method=cv2.RANSAC, threshold=1e-3)
    inliers = inliers.ravel().astype(bool)
    
    # recover pose
    n_in, R, t,_ = cv2.recoverPose(E, x1[inliers,:2], x2[inliers,:2])

    # compute camera matrices
    P_1 = np.array([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,0]])
    P_2 = np.hstack((R, t))
    
    # convert image to camera coordinates
    P_1c = K_cam @ P_1
    P_2c = K_cam @ P_2
        
    # triangulate using linear solution for each point in x1, and x2
    point_cloud = []
    for a,b in zip(x1, x2):
        t_point = triangulate(P_1c, P_2c, a, b)
        t_point /= t_point[-1] 
        point_cloud.append(t_point[:3])
    return point_cloud

In [13]:
def triangulate(P0,P1,x1,x2):
    # P0,P1: projection matrices for each of two cameras/images
    # x1,x1: corresponding points in each of two images (If using P that has been scaled by K, then use camera
    # coordinates, otherwise use generalized coordinates)
    A = np.array([[P0[2,0]*x1[0] - P0[0,0], P0[2,1]*x1[0] - P0[0,1], P0[2,2]*x1[0] - P0[0,2], P0[2,3]*x1[0] - P0[0,3]],
                  [P0[2,0]*x1[1] - P0[1,0], P0[2,1]*x1[1] - P0[1,1], P0[2,2]*x1[1] - P0[1,2], P0[2,3]*x1[1] - P0[1,3]],
                  [P1[2,0]*x2[0] - P1[0,0], P1[2,1]*x2[0] - P1[0,1], P1[2,2]*x2[0] - P1[0,2], P1[2,3]*x2[0] - P1[0,3]],
                  [P1[2,0]*x2[1] - P1[1,0], P1[2,1]*x2[1] - P1[1,1], P1[2,2]*x2[1] - P1[1,2], P1[2,3]*x2[1] - P1[1,3]]])
    u,s,vt = np.linalg.svd(A)
    return vt[-1]

In [28]:
img1 = plt.imread("./pictures/cube/DSC03896.JPG")
img2 = plt.imread("./pictures/cube/DSC03897.JPG")
img3 = plt.imread("./pictures/cube/DSC03898.JPG")

u11, u21 = get_best_sift_matches(img1, img2, r=0.7)
p_cloud = get_correspondence_points(img1, "./pictures/cube/DSC03896.JPG", u1, u2)
    
u22, u31 = get_best_sift_matches(img2, img3, r=0.7)
# set of keypoints across all three images
# from generalized coordinates x1:x2 and x2:x3, find shared matches
h,w,d = img1.shape
exif = piexif.load("./pictures/cube/DSC03896.JPG")
f = exif['Exif'][piexif.ExifIFD.FocalLengthIn35mmFilm]/36*w
cu = w//2
cv = h//2

K_cam = np.array([[f,0,cu], [0,f,cv], [0,0,1]])
K_inv = np.linalg.inv(K_cam)
x1 = u11 @ K_inv.T
x21 = u21 @ K_inv.T

h,w,d = img2.shape
exif = piexif.load("./pictures/cube/DSC03897.JPG")
f = exif['Exif'][piexif.ExifIFD.FocalLengthIn35mmFilm]/36*w
cu = w//2
cv = h//2

K_cam = np.array([[f,0,cu], [0,f,cv], [0,0,1]])
K_inv = np.linalg.inv(K_cam)
x22 = u22 @ K_inv.T
x3 = u31 @ K_inv.T            

In [30]:
matched_points = []
for a in x21:
    for b in x22:
        if np.allclose(a,b):
            matched_points.append(a)
            break
# matched_points = np.allclose(x21, x22)
m = np.array(matched_points)
print(m)

[[ -1.72266574e-01  -4.56497260e-02   1.00000000e+00]
 [ -2.02059747e-01   2.55647786e-02   1.00000000e+00]
 [ -2.01151666e-01   2.56175673e-02   1.00000000e+00]
 [ -2.01151666e-01   2.56175673e-02   1.00000000e+00]
 [ -1.55282104e-01  -4.46004232e-03   1.00000000e+00]
 [ -1.33908664e-01  -2.05297309e-02   1.00000000e+00]
 [ -1.27410075e-01   1.29425320e-02   1.00000000e+00]
 [ -1.38416477e-01   4.17081977e-02   1.00000000e+00]
 [ -1.21124227e-01   2.89299995e-02   1.00000000e+00]
 [ -1.56629137e-01   8.16309136e-02   1.00000000e+00]
 [ -6.94511447e-02   4.48391385e-03   1.00000000e+00]
 [ -2.38761664e-02  -8.85353122e-02   1.00000000e+00]
 [ -7.68374023e-02   3.24269748e-02   1.00000000e+00]
 [ -1.22400160e-01   8.46972114e-02   1.00000000e+00]
 [ -1.22400160e-01   8.46972114e-02   1.00000000e+00]
 [ -3.39924859e-02  -6.86044108e-02   1.00000000e+00]
 [ -8.71757270e-02   5.30390896e-02   1.00000000e+00]
 [ -8.71757270e-02   5.30390896e-02   1.00000000e+00]
 [ -8.71757270e-02   5.30390